In [ ]:
"""
Created on Fri Jan 27 16:14 2023

This script is to plot T and S profiles from both ensembles

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import basal_melt_param.melt_functions as meltf
from basal_melt_param.constants import *
import basal_melt_param.T_S_profile_functions as tspf
import seaborn as sns
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import cmocean
import glob

In [ ]:
%matplotlib qt5

READ DATA

In [ ]:
plot_path = '/bettik/burgardc/PLOTS/NN_plots/input_vars/'
outputpath_simple_all = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/SIMPLE/nemo_5km_06161821/'


In [ ]:
isf_dim = [10, 11, 12, 13, 18, 22, 23, 24, 25, 30, 31, 33, 38, 39, 40, 42, 43, 44, 45, 47, 48, 51, 52, 53, 54, 55, 58, 61, 65, 66, 69, 70, 71, 73, 75]

In [ ]:
run_list = ['OPM021','OPM018','OPM016','OPM006'] #,'OPM026','OPM027','OPM031'] 
#run_list = ['OPM021'] 
file_isf_list = []
TS_profile_mean_list = []
TS_profile_std_list = []
TS_profile_list = []
plume_list = []

for n,nemo_run in enumerate(run_list):
    
    print(nemo_run)
    
    # File mask
    inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    file_isf_list.append(file_isf)

    inputpath_profiles = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
    file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018_oneFRIS.nc')
    if nemo_run == 'OPM006':
        file_TS_otherdepth = file_TS_orig.sel(Nisf=large_isf, profile_domain=50)
        file_TS = file_TS_otherdepth.interp({'depth': TS_profile_mean_list[0].depth})
    else:
        file_TS = file_TS_orig.sel(Nisf=large_isf, profile_domain=50)
    file_TS_mean = file_TS.mean('time')
    file_TS_std = file_TS.std('time')
    TS_profile_mean_list.append(file_TS_mean)
    TS_profile_std_list.append(file_TS_std)
    TS_profile_list.append(file_TS.assign_coords({'nemo_run': nemo_run}))
    
    inputpath_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
    plume_charac = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS.nc')
    plume_list.append(plume_charac)
    
file_isf_all = xr.concat(file_isf_list, dim='nemo_run')

TS_profile_mean = xr.concat(TS_profile_mean_list, dim='nemo_run')
TS_profile_std = xr.concat(TS_profile_std_list, dim='nemo_run')
TS_profile_all = xr.concat(TS_profile_list, dim='nemo_run')
plume_mathiot =  xr.concat(plume_list, dim='nemo_run')

In [ ]:
TS_profile_all

In [ ]:
TS_profile_all['theta_ocean'].sel(Nisf=66).isel(nemo_run=1).plot()

In [ ]:
inputpath_profiles = '/bettik/burgardc/DATA/NN_PARAM/interim/T_S_PROF/SMITH_bi646/'
inputpath_plumes = '/bettik/burgardc/DATA/NN_PARAM/interim/PLUMES/SMITH_bi646/'

yy_list = range(1980,1970+71)
prof_list = []
plume_list = []
for yy in yy_list:
    file_TS_yy = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_'+str(yy)+'.nc')
    if 'labels' in file_TS_yy.coords.keys():
        file_TS_yy = file_TS_yy.drop('labels')
    prof_list.append(file_TS_yy)
    
    plume_charac_yy = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS_'+str(yy)+'.nc')
    plume_list.append(plume_charac_yy)
    
file_TS_bi646 = xr.concat(prof_list, dim='time').assign_coords({'nemo_run': 'bi646'})
file_TS_mean_bi646 = file_TS_bi646.mean('time')
file_TS_std_bi646 = file_TS_bi646.std('time')
plume_bi646 = xr.concat(plume_list, dim='time').assign_coords({'nemo_run': 'bi646'})

In [ ]:
inputpath_profiles = '/bettik/burgardc/DATA/NN_PARAM/interim/T_S_PROF/SMITH_bf663/'
inputpath_plumes = '/bettik/burgardc/DATA/NN_PARAM/interim/PLUMES/SMITH_bf663/'

yy_list = range(1980,1970+71)
prof_list = []
plume_list = []
for yy in yy_list:
    file_TS_yy = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_'+str(yy)+'.nc')
    if 'labels' in file_TS_yy.coords.keys():
        file_TS_yy = file_TS_yy.drop('labels')
    prof_list.append(file_TS_yy)
    
    plume_charac_yy = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS_'+str(yy)+'.nc')
    plume_list.append(plume_charac_yy)
    
file_TS_bf663 = xr.concat(prof_list, dim='time').assign_coords({'nemo_run': 'bf663'})
file_TS_mean_bf663 = file_TS_bf663.mean('time')
file_TS_std_bf663 = file_TS_bf663.std('time')
plume_bf663 = xr.concat(plume_list, dim='time').assign_coords({'nemo_run': 'bf663'})

In [ ]:
sorted_isf_all = [11,69,43,12,70,44,29,13,58,71,45,30,31,61,73,47,32,48,33,17,62,49,34,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,26,42,55]

In [ ]:
inputpath_mask = '/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_bf663/'
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_1972.nc')
file_isf = file_isf_orig.sel(Nisf=file_TS_mean_bf663.Nisf).squeeze().drop('time')

In [ ]:
file_TS_mean_smith = xr.concat([file_TS_mean_bf663, file_TS_mean_bi646], dim='nemo_run')
file_TS_std_smith = xr.concat([file_TS_std_bf663, file_TS_std_bi646], dim='nemo_run')

In [ ]:
file_isf.Nisf

ONLY SMITH PROFILES

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','Eastern Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','brown','magenta','yellow','cyan','blue','magenta']

reg_list = file_isf['region'].astype(str)

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0
for rr,reg in enumerate(regions):
    
    print(reg)
    
    subset_isf = file_isf.Nisf.where(reg_list==reg,drop=True).drop('nemo_run')
    
    if len(subset_isf > 0):
        
        for kisf in tqdm(subset_isf.Nisf):

            mean_data = file_TS_mean_all['theta_ocean'].sel(Nisf=kisf)
            std_data = file_TS_std_all['theta_ocean'].sel(Nisf=kisf)

            ax[i] = f.add_subplot(6,6,i+1)

            for n, nrun in enumerate(mean_data.nemo_run):

                if nrun == 'bf663':
                    mean_run = file_TS_mean_bf663['theta_ocean'].sel(Nisf=kisf)
                    std_run = file_TS_std_bf663['theta_ocean'].sel(Nisf=kisf)
                elif nrun == 'bi646':
                    mean_run = file_TS_mean_bi646['theta_ocean'].sel(Nisf=kisf)
                    std_run = file_TS_std_bi646['theta_ocean'].sel(Nisf=kisf)

                plt.plot(mean_run,-1*mean_run.depth, color=colors[n], linewidth=3)
                #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
                plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
                #leg_hdl.append(leg)


            ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
            ax[i].set_xlim(-2.2,2.5)

            i = i+1
        

f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()

In [ ]:
file_isf_orig['isf_name'] = file_isf_orig['isf_name'].where(file_isf_orig['isf_name'] != 'Ekstrom', np.array('Ekström', dtype=object))

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','Eastern Ross','East 1','East 2','Dronning Maud Land']
colors = ['orange','red']

reg_list = file_isf['region'].astype(str)

alpha_val = 0.07

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

for kisf in tqdm(sorted_isf_all):

    ax[i] = f.add_subplot(7,7,i+1)

    if kisf in isf_dim:                

        for nrun in ['OPM006','OPM016','OPM018','OPM021']:

            for tt in TS_profile_all.sel(nemo_run = nrun).time:

                T_data = TS_profile_all['theta_ocean'].sel(nemo_run = nrun, time = tt, Nisf = kisf)
                ax[i].plot(T_data,-1*T_data.depth, color='grey', alpha=alpha_val)

    if kisf in list(file_isf.Nisf.values):

        for n, nrun in enumerate(['bf663','bi646']):

            for tt in range(1980, 1980 + 60):

                if nrun == 'bf663':

                    T_data = file_TS_bf663['theta_ocean'].sel(time = tt, Nisf = kisf)

                elif nrun == 'bi646':

                    T_data = file_TS_bi646['theta_ocean'].sel(time = tt, Nisf = kisf)

                ax[i].plot(T_data,-1*T_data.depth, color=colors[n], alpha=alpha_val)
        #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
        #plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
        #leg_hdl.append(leg)

    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    ax[i].set_xlim(-2.2,3)

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'T_profiles_all.png', dpi=300)

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','Eastern Ross','East 1','East 2','Dronning Maud Land']
colors = ['orange','red']

reg_list = file_isf['region'].astype(str)

alpha_val = 0.07

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

for kisf in tqdm(sorted_isf_all):

    ax[i] = f.add_subplot(7,7,i+1)

    if kisf in isf_dim:                

        for nrun in ['OPM006','OPM016','OPM018','OPM021']:

            for tt in TS_profile_all.sel(nemo_run = nrun).time:

                T_data = TS_profile_all['salinity_ocean'].sel(nemo_run = nrun, time = tt, Nisf = kisf)
                ax[i].plot(T_data,-1*T_data.depth, color='grey', alpha=alpha_val)

    if kisf in list(file_isf.Nisf.values):

        for n, nrun in enumerate(['bf663','bi646']):

            for tt in range(1980, 1980 + 60):

                if nrun == 'bf663':

                    T_data = file_TS_bf663['salinity_ocean'].sel(time = tt, Nisf = kisf)

                elif nrun == 'bi646':

                    T_data = file_TS_bi646['salinity_ocean'].sel(time = tt, Nisf = kisf)

                ax[i].plot(T_data,-1*T_data.depth, color=colors[n], alpha=alpha_val)
        #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
        #plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
        #leg_hdl.append(leg)

    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    ax[i].set_xlim(31,35)

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'S_profiles_all.png', dpi=300)

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','brown','magenta','yellow','cyan','blue','magenta']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0
for rr,reg in enumerate(regions):
    
    subset_isf = file_isf.Nisf.where(file_isf['region']==reg,drop=True)
    
    for kisf in tqdm(subset_isf.Nisf):

        mean_data = file_TS_mean_all['salinity_ocean'].sel(Nisf=kisf)
        std_data = file_TS_std_all['salinity_ocean'].sel(Nisf=kisf)
        
        ax[i] = f.add_subplot(6,6,i+1)
        
        for n, nrun in enumerate(mean_data.nemo_run):
            
            if nrun == 'bf663':
                mean_run = file_TS_mean_bf663['salinity_ocean'].sel(Nisf=kisf)
                std_run = file_TS_std_bf663['salinity_ocean'].sel(Nisf=kisf)
            elif nrun == 'bi646':
                mean_run = file_TS_mean_bi646['salinity_ocean'].sel(Nisf=kisf)
                std_run = file_TS_std_bi646['salinity_ocean'].sel(Nisf=kisf)
                
            plt.plot(mean_run,-1*mean_run.depth, color=colors[n], linewidth=3)
            #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
            plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
            #leg_hdl.append(leg)
            
        
        ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
        ax[i].set_xlim(32,35)
        
        i = i+1
        

f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()

In [ ]:
mean_run.plot()

In [ ]:
mean_data.isel(nemo_run=0).plot()